In [ ]:
import cv2
import numpy as np

# Cargar imagen de referencia (la señal que quieres detectar)
img_ref = cv2.imread('resbalar.jpg', cv2.IMREAD_GRAYSCALE)
if img_ref is None:
    print("No se encontró la imagen de referencia.")
    exit()

# Crear el detector SIFT
sift = cv2.SIFT_create()

# Detectar puntos clave y descriptores en la imagen de referencia
kp_ref, des_ref = sift.detectAndCompute(img_ref, None)

# Crear el matcher (match de características)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# Iniciar cámara
cap = cv2.VideoCapture(0)

# Umbrales de tamaño en píxeles del ancho del objeto detectado
pix_cerca = 250  # Ancho en píxeles que indica que el objeto está muy cerca
pix_medio = 150  # Umbral intermedio para empezar a acercarse

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar puntos clave y descriptores en la imagen de la cámara
    kp_frame, des_frame = sift.detectAndCompute(gray, None)
    if des_frame is None or len(kp_frame) == 0:
        continue

    # Hacer el matching de descriptores
    matches = bf.knnMatch(des_ref, des_frame, k=2)

    # Filtrar las buenas coincidencias usando la relación de Lowe
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) >= 10:
        pts_ref = np.float32([kp_ref[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        pts_frame = np.float32([kp_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Calcular la homografía
        H, mask = cv2.findHomography(pts_ref, pts_frame, cv2.RANSAC, 5.0)

        if H is not None:
            h, w = img_ref.shape
            pts_ref_corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
            pts_frame_corners = cv2.perspectiveTransform(pts_ref_corners, H)

            # Dibujar el polígono de la señal detectada en la imagen
            frame = cv2.polylines(frame, [np.int32(pts_frame_corners)], True, (0, 255, 0), 3)

            # Obtener ancho del objeto detectado en píxeles
            w_obj = np.linalg.norm(pts_frame_corners[0] - pts_frame_corners[1])
            cv2.putText(frame, f"Ancho px: {int(w_obj)}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            # Evaluar centrado
            center_frame_x = frame.shape[1] / 2
            cx = np.mean([pt[0][0] for pt in pts_frame_corners])

            if abs(cx - center_frame_x) < 30:
                centrado = True
                cv2.putText(frame, "SENAL CENTRADA", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                centrado = False
                if cx < center_frame_x:
                    texto = "GIRAR A LA IZQUIERDA"
                else:
                    texto = "GIRAR A LA DERECHA"
                cv2.putText(frame, texto, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Evaluar si acercarse o detenerse
            if w_obj > pix_cerca:
                estado = "MUY CERCA - DETENERSE"
            elif w_obj > pix_medio:
                estado = "ACERCANDOSE..."
            else:
                estado = "OBJETO LEJOS"

            cv2.putText(frame, estado, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
            print(estado)

    # Mostrar la imagen con los resultados
    cv2.imshow("Deteccion", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS
OBJETO LEJOS